        Data Fields

**[] fullVisitorId**- A unique identifier for each user of the Google Merchandise Store.

**[x] channelGrouping** - The channel via which the user came to the Store. 

**[x] date** - The date on which the user visited the Store.

**[x] device** - The specifications for the device used to access the Store.

**[x] geoNetwork** - This section contains information about the geography of the user.

**[] sessionId** - A unique identifier for this visit to the store.

**[x] socialEngagementType** - Engagement type, either "Socially Engaged" or "Not Socially Engaged".

**[x] totals** - This section contains aggregate values across the session.

**[x] trafficSource** - This section contains information about the Traffic Source from which the session originated.

**[] visitId** - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.

**[] visitNumber** - The session number for this user. If this is the first session, then this is set to 1.

**[] visitStartTime** - The timestamp (expressed as POSIX time).

In [ ]:
# Import libraries
import pandas as pd
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import xgboost as xgb
import lightgbm as lgb
import catboost as cat

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
from itertools import cycle

from sklearn.model_selection import train_test_split

**Function to read data from JSON format**

In [ ]:
def load_df(csv_path='../input/train-v1/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../input/train-v1"))

In [ ]:
%%time
train_origin = load_df()
test_origin = load_df("../input/train-v1/test.csv")

In [ ]:
train_origin.shape, test_origin.shape

In [ ]:
train_origin.columns

**Analyzing target**

The target we want to predict, transactionRevenue, is contained in one of the JSON columns, ie. the totals column. While loading the dataset, it was renamed as totals.transactionRevenue. The target only contains a few non-null values and before taking its log, we fill the NAs:

In [ ]:
target = train_origin['totals.transactionRevenue'].fillna(0).astype(float)
print(np.mean(target))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

sns.set(color_codes=True)
sns.distplot(target[target > 0], kde=False, rug=True);

**Fill Nan Value for all other attributes**

In [ ]:
columns_to_remove = [col for col in train_origin.columns if train_origin[col].nunique() == 1]
print("Nb. of variables with unique value: {}".format(len(columns_to_remove)))

for col in columns_to_remove:
    if set(['not available in demo dataset']) ==  set(train_origin[col].unique()): continue
    print(col, train_origin[col].dtypes, train_origin[col].unique())

In [ ]:
# Do not remove columns with unique value and NAN, update value for NAN
train_origin['totals.bounces'] = train_origin['totals.bounces'].fillna('0')
test_origin['totals.bounces'] = test_origin['totals.bounces'].fillna('0')

train_origin['totals.newVisits'] = train_origin['totals.newVisits'].fillna('0')
test_origin['totals.newVisits'] = test_origin['totals.newVisits'].fillna('0')

train_origin['trafficSource.adwordsClickInfo.isVideoAd'] = train_origin['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)
test_origin['trafficSource.adwordsClickInfo.isVideoAd'] = test_origin['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True)

train_origin['trafficSource.isTrueDirect'] = train_origin['trafficSource.isTrueDirect'].fillna(False)
test_origin['trafficSource.isTrueDirect'] = test_origin['trafficSource.isTrueDirect'].fillna(False)


**Clear rare categories**

In [ ]:
# Remove column not in test but in train, should we?
print("Columns in Train but NOT in Test:")
for col in train_origin.columns:
    if col not in test_origin.columns:
        print(col)
print("Columns in Test but NOT in Traing:")
for col in test_origin.columns:
    if col not in train_origin.columns:
        print(col)
train_origin['trafficSource.campaignCode'].value_counts()
# trafficSource.campaignCode is redundant, we will not use for training/testing

In [ ]:
def clearRare(columnname, limit = 100):
    # you may search for rare categories in train, train&test, or just test
    #vc = pd.concat([train[columnname], test[columnname]], sort=False).value_counts()
    vc = test_origin[columnname].value_counts()
    
    common = vc > limit
    common = set(common.index[common].values)
    print("Set", sum(vc <= limit), columnname, "categories to 'other';", end=" ")

    train_origin.loc[train_origin[columnname].map(lambda x: x not in common), columnname] = 'other'
    test_origin.loc[test_origin[columnname].map(lambda x: x not in common), columnname] = 'other'
    print("now there are", train_origin[columnname].nunique(), "categories in train")

**Fill NAN exhausted**

In [ ]:
train_origin.fillna(0, inplace=True)
test_origin.fillna(0, inplace=True)

In [ ]:
clearRare("device.browser")
clearRare("device.operatingSystem")
clearRare("geoNetwork.country")
clearRare("geoNetwork.city")
clearRare("geoNetwork.metro")
clearRare("geoNetwork.networkDomain")
clearRare("geoNetwork.region")
clearRare("geoNetwork.subContinent")
clearRare("trafficSource.adContent")
clearRare("trafficSource.campaign")
clearRare("trafficSource.keyword")
clearRare("trafficSource.medium")
clearRare("trafficSource.referralPath")
clearRare("trafficSource.source")

In [ ]:
test_origin.shape, train_origin.shape

**Convert Date into Understandable format**

In [ ]:
for df in [train_origin, test_origin]:
    df_date = pd.to_datetime(df['visitStartTime'], unit='s')
    df['sess_date_dow'] = df_date.dt.dayofweek
    df['sess_date_hours'] = df_date.dt.hour
    df['sess_date_dom'] = df_date.dt.day
train_origin.shape, test_origin.shape

**Analyzing transaction time of dataset**

In [ ]:
train_date = pd.to_datetime(train_origin.date, format="%Y%m%d")
test_date = pd.to_datetime(test_origin.date, format="%Y%m%d")
train_date.value_counts().sort_index().plot(label="train")
test_date.value_counts().sort_index().plot(label="test")
# plt.legend()

In [ ]:
# distribution by day of week
sns.set(color_codes=True)
sns.distplot(train_origin['sess_date_dow'], kde=False, rug=True);

In [ ]:
# Show date of week when people have made transactions
dateWeek = train_origin[train_origin['totals.transactionRevenue'] != 0]
sns.distplot(dateWeek['sess_date_dow'], kde=False, rug=True);

In [ ]:
# distribution by hour of day
sns.distplot(train_origin['sess_date_hours'], kde=False, rug=True);

In [ ]:
# Show hours when people have made transactions
hourDate = train_origin[train_origin['totals.transactionRevenue'] != 0]
sns.distplot(hourDate['sess_date_hours'], kde=False, rug=True);

In [ ]:
# distribution by day of month
sns.distplot(train_origin['sess_date_dom'], kde=False, rug=True);

In [ ]:
# Show date of month when people have done transactions
dateMonth = train_origin[train_origin['totals.transactionRevenue'] != 0]
sns.distplot(dateMonth['sess_date_dom'], kde=False, rug=True);

In [ ]:
for col in train_origin.columns:
    print(col, train_origin[col].dtype)

**Create feature list**

In [ ]:
excluded_features = [
    'date', 'sessionId', 'fullVisitorId', 
    'totals.transactionRevenue', 'trafficSource.campaignCode',
    'visitId', 'visitStartTime','visitNumber'
]

no_object_columns = ['device.isMobile', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect',
                    'sess_date_dow', 'sess_date_hours', 'sess_date_dom']

categorical_features = [
    _f for _f in train_origin.columns
    if (_f not in excluded_features) # & (train_df[_f].dtype == 'object')
]
print("Number of selected features : ",len(categorical_features))

**Final columns after filter columns with unique values**

In [ ]:
columns = [col for col in train_origin.columns if  train_origin[col].nunique() > 1 and (col in categorical_features)]
print(len(columns))
print(columns)

**DONE for select feature and factorizing dataset**

**DATA Visualization**

In [ ]:
train_origin['geoNetwork.continent'].value_counts()

In [ ]:
# Analysis total revenue by continent
ttt = train_origin[train_origin['totals.transactionRevenue'] != 0].copy()
ttt['totals.transactionRevenue'] = ttt['totals.transactionRevenue'].astype(float)
ttt.groupby(by='geoNetwork.continent')['totals.transactionRevenue'].sum()

In [ ]:
def drawBars(columnname):
    sns.barplot(x="count", y="index", hue="dataset",
        data=pd.melt(pd.concat([train_origin[columnname].value_counts().rename("train"), 
                       test_origin[columnname].value_counts().rename("test")], axis=1, sort="False").reset_index(),
            id_vars="index", var_name="dataset", value_name="count"))

In [ ]:
#find country where total_transactionRevenue > 0
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
def plot_countryMap(data, country, nvisits, visits, title, colormap='Viridis'):
    data = dict(type = 'choropleth', 
                colorscale = colormap,
                autocolorscale = False,
                reversescale = False,
               locations = data[country],
               locationmode = 'country names',
               z = data[nvisits], 
               text = data[nvisits],
               colorbar = {'title':visits})
    layout = dict(title = title, 
                 geo = dict(showframe = False, 
                         projection = {'type': 'natural earth'}))
    choromap = go.Figure(data = [data], layout=layout)
    iplot(choromap)

In [ ]:
#visits per country
countries = train_origin['geoNetwork.country'].value_counts()
countryVisits = pd.DataFrame(data={'geoNetwork_country': countries.values}, index=countries.index).reset_index()
countryVisits.columns = ['Country', 'Visits']
plot_countryMap(countryVisits, 'Country', 'Visits', 'Visits', 'Visits per country')

In [ ]:
drawBars("channelGrouping")

In [ ]:
#visits with non-zero transaction
nonZero = train_origin[train_origin['totals.transactionRevenue'] != 0].copy()
countries = nonZero['geoNetwork.country'].value_counts()
countryVisits = pd.DataFrame(data={'geoNetwork_country': countries.values}, index=countries.index).reset_index()
countryVisits.columns = ['Country', 'Visits']
plot_countryMap(countryVisits, 'Country', 'Visits', 'Visits', 'Visits with non-zero transaction per country')

In [ ]:
ids_train = set(train_origin.fullVisitorId.unique())
ids_test = set(test_origin.fullVisitorId.unique())
print("Unique visitor ids in train:", len(ids_train))
print("Unique visitor ids in test:", len(ids_test))
print("Common visitors in train and test:", len(ids_train & ids_test))

In [ ]:
print("Train: ", np.bincount(train_origin.visitId.value_counts()))
print("test: ", np.bincount(test_origin.visitId.value_counts()))


**Factorization**

In [ ]:
object_columns = [i for i in columns if i not in no_object_columns]
for f in object_columns:
    print(f)
    train_origin[f], indexer = pd.factorize(train_origin[f])
    test_origin[f] = indexer.get_indexer(test_origin[f])
train_origin["totals.transactionRevenue"] = train_origin["totals.transactionRevenue"].fillna(0).astype(int)
train_origin["totals.transactionRevenue"] = train_origin["totals.transactionRevenue"].astype('int')
train_origin.head(10)

**Build Models**

**Classify non-zero revenues**

There are only 1.3% of the sessions have a non-zero revenue. Thus, we first CLASSIFY session, and then using regression on non-zero set.

In [ ]:
train_df = train_origin.copy()
test_df = test_origin.copy()

In [ ]:
from sklearn.model_selection import train_test_split
x = train_df[columns]
y = train_df["totals.transactionRevenue"]
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
y_valid[y_valid > 0].shape, y_train[y_train > 0].shape

In [ ]:
# Transfer target into format of [0, 1]
y_train01 = y_train.copy()
y_train01[y_train01 > 0] = 1

y_valid01 = y_valid.copy()
y_valid01[y_valid01 > 0] = 1

In [ ]:
# Ratio of classification 0, 1
y_train01.value_counts() / y_train01.shape[0]

**Visualize data distribution using PCA**

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2).fit(x)
X_reduced = pca.transform(x)

yy = y.copy()
yy[yy>0] = 1

plt.clf()
fig, ax = plt.subplots()

# Plot the training points
labels = ['0','1']
colors = ['white','red']
for i in range(2):   
    ax.scatter(X_reduced[yy==i, 0], X_reduced[yy==i, 1], c=colors[i],label = labels[i],cmap=plt.cm.Set1,s=40,
                edgecolor=None,alpha=0.3)


plt.title("First two PCA directions")
plt.xlabel("1st PC")
plt.ylabel("2nd PC")
ax.legend()
plt.xticks(())
plt.yticks(())

**XGBoost Classifier**

*Paramters tuning:* learning rate,
n estimators,
min child weight,
max delta step, booster,
seed

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb = XGBClassifier()
xgb_param = {'n_estimators':[100, 500, 1000]}

clf = GridSearchCV(xgb, xgb_param, cv=5, verbose=True, scoring='f1')
clf.fit(x_train,y_train01)

In [ ]:
clf.best_estimator_

In [ ]:
y_pred = clf.predict(x_train)
y_pred_valid = clf.predict(x_valid)
print("f1 score:")
print("Train data: " + str(f1_score(y_train01, y_pred)))
print("Test data: " + str(f1_score(y_valid01, y_pred_valid)))

print()
print("Accuracy Score:")
print("Train data: " + str(accuracy_score(y_train01, y_pred)))
print("Test data: " + str(accuracy_score(y_valid01, y_pred_valid)))
print()

print("ROC AUC Score:")
print("Train data: " + str(roc_auc_score(y_train01, y_pred)))
print("Test data: " + str(roc_auc_score(y_valid01, y_pred_valid)))
print()

#Confusion Matrix
print("Confusion Matrix")
y_actual = pd.Series(y_valid01, name='Actual')
y_predict = pd.Series(y_pred_valid, name='Predicted')
df_confusion = pd.crosstab(y_actual, y_predict)
print(df_confusion)

plt.figure(figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
plt.matshow(df_confusion, cmap='viridis')
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(df_confusion.columns))
plt.xticks(tick_marks, df_confusion.columns, rotation=45)
plt.yticks(tick_marks, df_confusion.index)
plt.ylabel(df_confusion.index.name)
plt.xlabel(df_confusion.columns.name)

In [ ]:
# Predict test
y_submit_xgb = clf.predict(test_df[columns])
y_submit_xgb[y_submit_xgb > 0] = 10272.0
pd.Series(y_submit_xgb).value_counts()

In [ ]:
# Prediction
y_submit = clf.predict(test_df[columns])

In [ ]:
#Predicted revenue per customer
submission = test_origin[['fullVisitorId']].copy()
submission.loc[:, 'PredictedLogRevenue'] = y_submit
grouped_test_RF = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
grouped_test_RF["PredictedLogRevenue"] = np.log1p(grouped_test_RF["PredictedLogRevenue"])
print(grouped_test_RF)

**Random Forest**

*Tuning paramters*
n_estimators,
criterion,
max_depth,
max_features,
min_samples_split,
min_samples_leaf

In [ ]:
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
rf_param = {'n_estimators':[100, 500, 1000]}

clf_rf = GridSearchCV(rf,rf_param,cv=5,verbose=True,scoring='f1')
clf_rf.fit(x_train,y_train01)

In [ ]:
clf_rf.best_estimator_

In [ ]:
y_pred = clf_rf.predict(x_train)
y_pred_valid = clf_rf.predict(x_valid)

print("f1 score:")
print("Train data: " + str(f1_score(y_train01, y_pred)))
print("Test data: " + str(f1_score(y_valid01, y_pred_valid)))

print()
print("Accuracy Score:")
print("Train data: " + str(accuracy_score(y_train01, y_pred)))
print("Test data: " + str(accuracy_score(y_valid01, y_pred_valid)))
print()

print("ROC AUC Score:")
print("Train data: " + str(roc_auc_score(y_train01, y_pred)))
print("Test data: " + str(roc_auc_score(y_valid01, y_pred_valid)))
print()

#Confusion Matrix
print("Confusion Matrix")
y_actual = pd.Series(y_valid01, name='Actual')
y_predict = pd.Series(y_pred_valid, name='Predicted')
df_confusion = pd.crosstab(y_actual, y_predict)
print(df_confusion)

plt.figure(figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
plt.matshow(df_confusion, cmap='viridis')
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(df_confusion.columns))
plt.xticks(tick_marks, df_confusion.columns, rotation=45)
plt.yticks(tick_marks, df_confusion.index)
plt.ylabel(df_confusion.index.name)
plt.xlabel(df_confusion.columns.name)

In [ ]:
# Predict test
y_submit_rf = clf_rf.predict(test_df[columns])
y_submit_rf[y_submit_rf > 0] = 10272.0
pd.Series(y_submit_rf).value_counts()

In [ ]:
# Prediction
y_submit = clf_rf.predict(test_df[columns])

In [ ]:
#Predicted Revenue per customer output
submission = test_origin[['fullVisitorId']].copy()
submission.loc[:, 'PredictedLogRevenue'] = y_submit
grouped_test_RF = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
grouped_test_RF["PredictedLogRevenue"] = np.log1p(grouped_test_RF["PredictedLogRevenue"])
print(grouped_test_RF)

**Ensemble learning of classifiers RF + XGB**

In [ ]:
y_combination = (y_submit_rf + y_submit_xgb) / 2

In [ ]:
print("Confusion Matrix")
y_actual = pd.Series(y_valid01, name='Actual')
y_predict = pd.Series(y_combination, name='Predicted')
df_confusion = pd.crosstab(y_actual, y_predict)
print(df_confusion)

plt.figure(figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
plt.matshow(df_confusion, cmap='viridis')
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(df_confusion.columns))
plt.xticks(tick_marks, df_confusion.columns, rotation=45)
plt.yticks(tick_marks, df_confusion.index)
plt.ylabel(df_confusion.index.name)
plt.xlabel(df_confusion.columns.name)

In [ ]:
#Predicted revenue per customer
submission = test_origin[['fullVisitorId']].copy()
submission.loc[:, 'PredictedLogRevenue'] = y_combination
grouped_test_RF = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
grouped_test_RF["PredictedLogRevenue"] = np.log1p(grouped_test_RF["PredictedLogRevenue"])
grouped_test_RF.to_csv('submit_Clf.csv',index=False)
print(grouped_test_RF)

**Predict revenues for non-zero**

In [ ]:
train_origin[columns].head()

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error

def score(data, y):
    validation_res = pd.DataFrame(
    {"fullVisitorId": data["fullVisitorId"].values,
     "transactionRevenue": data["totals.transactionRevenue"].values,
     "predictedRevenue": np.expm1(y)})

    validation_res = validation_res.groupby("fullVisitorId")["transactionRevenue", "predictedRevenue"].sum().reset_index()
    return np.sqrt(mean_squared_error(np.log1p(validation_res["transactionRevenue"].values), 
                                     np.log1p(validation_res["predictedRevenue"].values)))

class KFoldValidation():
    def __init__(self, data, n_splits=5):
        unique_vis = np.array(sorted(data['fullVisitorId'].astype(str).unique()))
        folds = GroupKFold(n_splits)
        ids = np.arange(data.shape[0])
        
        self.fold_ids = []
        for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
            self.fold_ids.append([
                    ids[data['fullVisitorId'].astype(str).isin(unique_vis[trn_vis])],
                    ids[data['fullVisitorId'].astype(str).isin(unique_vis[val_vis])]
                ])
            
    def validate(self, train, test, features, model, name="", prepare_stacking=False, 
                 fit_params={"early_stopping_rounds": 50, "verbose": 100, "eval_metric": "rmse"}):
        model.FI = pd.DataFrame(index=features)
        full_score = 0
        
        if prepare_stacking:
            test[name] = 0
            train[name] = np.NaN
        
        for fold_id, (trn, val) in enumerate(self.fold_ids):
            devel = train[features].iloc[trn]
            y_devel = np.log1p(train["totals.transactionRevenue"].iloc[trn])
            valid = train[features].iloc[val]
            y_valid = np.log1p(train["totals.transactionRevenue"].iloc[val])
                       
            print("Fold ", fold_id, ":")
            model.fit(devel, y_devel, eval_set=[(valid, y_valid)], **fit_params)
            
            if len(model.feature_importances_) == len(features):  # some bugs in catboost?
                model.FI['fold' + str(fold_id)] = model.feature_importances_ / model.feature_importances_.sum()

            predictions = model.predict(valid)
            predictions[predictions < 0] = 0
            print("Fold ", fold_id, " error: ", mean_squared_error(y_valid, predictions)**0.5)
            
            fold_score = score(train.iloc[val], predictions)
            full_score += fold_score / len(self.fold_ids)
            print("Fold ", fold_id, " score: ", fold_score)
            
            if prepare_stacking:
                train[name].iloc[val] = predictions
                
                test_predictions = model.predict(test[features])
                test_predictions[test_predictions < 0] = 0
                test[name] += test_predictions / len(self.fold_ids)
        print("Confusion Matrix")
        y_actual = pd.Series(y_valid, name='Actual')
        y_predict = pd.Series(predictions, name='Predicted')
        df_confusion = pd.crosstab(y_actual, y_predict)
        print(df_confusion)

        plt.figure(figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
        plt.matshow(df_confusion, cmap='viridis')
        plt.title('Confusion matrix')
        plt.colorbar()
        tick_marks = np.arange(len(df_confusion.columns))
        plt.xticks(tick_marks, df_confusion.columns, rotation=45)
        plt.yticks(tick_marks, df_confusion.index)
        plt.ylabel(df_confusion.index.name)
        plt.xlabel(df_confusion.columns.name)        
        print("Final score: ", full_score)
        return full_score

In [ ]:
Kfolder = KFoldValidation(train_df)

**LGBMRegressor**

In [ ]:
lgbmodel = lgb.LGBMRegressor(n_estimators=1000, objective="regression", metric="rmse", num_leaves=31, min_child_samples=100,
                      learning_rate=0.03, bagging_fraction=0.7, feature_fraction=0.5, bagging_frequency=5, 
                      bagging_seed=2019, subsample=.9, colsample_bytree=.9, use_best_model=True)

In [ ]:
Kfolder.validate(train_df, test_df, columns, lgbmodel, "lgbpred", prepare_stacking=True)

In [ ]:
# Prediction
y_submit = lgbmodel.predict(test_df[columns])

In [ ]:
# output to file
submission = test_origin[['fullVisitorId']].copy()
submission.loc[:, 'PredictedLogRevenue'] = y_submit
grouped_test_LGB = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
grouped_test_LGB["PredictedLogRevenue"] = np.log1p(grouped_test_LGB["PredictedLogRevenue"])
grouped_test_LGB["PredictedLogRevenue"] = grouped_test_LGB["PredictedLogRevenue"].fillna(0).astype(float)
grouped_test_LGB.to_csv('submit_reg_lgb.csv',index=False)
test_df.head()

**Cat Model**


In [ ]:
catmodel = cat.CatBoostRegressor(iterations=1000, learning_rate=0.2, depth=5, random_seed=2019)

In [ ]:
Kfolder.validate(train_df, test_df, columns, catmodel, name="catpred", prepare_stacking=True,
                fit_params={"use_best_model": True, "verbose": 100})

In [ ]:
# Prediction
y_submit = catmodel.predict(test_df[columns])

In [ ]:
# output to file
submission = test_origin[['fullVisitorId']].copy()
submission.loc[:, 'PredictedLogRevenue'] = y_submit
grouped_test_CAT = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
grouped_test_CAT["PredictedLogRevenue"] = np.log1p(grouped_test_CAT["PredictedLogRevenue"])
grouped_test_CAT["PredictedLogRevenue"] = grouped_test_CAT["PredictedLogRevenue"].fillna(0).astype(float)
grouped_test_CAT.to_csv('submit_reg_cat.csv',index=False)

In [ ]:
test_df.head()

**Ensemble learning **

In this part, we use result of models (LGB, CAT) for ensemble learning

In [ ]:
train_df['PredictedLogRevenue'] = 0.6 * train_df["lgbpred"] + 0.4 * train_df["catpred"] 
score(train_df, train_df.PredictedLogRevenue)

In [ ]:
test_df['PredictedLogRevenue'] = 0.6 * test_df["lgbpred"] +  0.4 * test_df["catpred"]
y_submit =  test_df['PredictedLogRevenue']

In [ ]:
# output to file
submission = test_origin[['fullVisitorId']].copy()
submission.loc[:, 'PredictedLogRevenue'] = y_submit
grouped_test_ENS = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
grouped_test_ENS["PredictedLogRevenue"] = np.log1p(grouped_test_ENS["PredictedLogRevenue"])
grouped_test_ENS["PredictedLogRevenue"] = grouped_test_ENS["PredictedLogRevenue"].fillna(0).astype(float)
grouped_test_ENS.to_csv('submit_reg_ens.csv',index=False)
grouped_test_ENS.head()

**======== THE END ========**